# Submission file hard voting

- `./hard_voting` 폴더 내에 voting 하고자 하는 csv 파일 삽입 (개수 제한 X)
- hard_voting 된 결과는 동일 폴더내에 `submission_voted.csv` 로 저장

In [81]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

In [82]:
output_file_list = glob("./*.csv")
output_file_list

['./output (2).csv',
 './output (10).csv',
 './output (7).csv',
 './output (8).csv',
 './output (1).csv',
 './output (5).csv',
 './output (9).csv',
 './output (6).csv',
 './output (3).csv',
 './output (4).csv']

In [83]:
df_list = list()
for file in output_file_list:
    answer = pd.read_csv(file)
    df_list.append(answer)

In [84]:
user_recommends = dict()

for df_index in tqdm(np.arange(len(df_list))): # 10개 기준 30초
    group_user = df_list[df_index].groupby("user")["item"]
    for user in group_user.groups.keys():
        user_recommended10 = group_user.get_group(user)
        if user in user_recommends:
            user_recommends[user].extend(user_recommended10.tolist())
        else:
            user_recommends[user] = user_recommended10.tolist()

100%|██████████| 10/10 [00:33<00:00,  3.36s/it]


In [85]:
from collections import Counter

final_recommends = dict()
for user in tqdm(user_recommends.keys()):
    count_result = Counter(user_recommends[user])
    most_common_list = list(count_result.most_common())
    final_recommends[user] = [x[0] for x in most_common_list[:10]]

100%|██████████| 31360/31360 [00:00<00:00, 65826.16it/s]


In [86]:
# 모든 유저에 대해 ensemble증명
assert len(final_recommends.keys()) == 31360

In [93]:
submission_list = list()

for user in tqdm(final_recommends.keys()): # 0초
    for item in final_recommends[user]:
        submission_list.append([user, item])

100%|██████████| 31360/31360 [00:00<00:00, 48303.85it/s]


In [96]:
submission_df = pd.DataFrame(data=submission_list, columns=["user", "item"])
submission_df.to_csv("./submission_voted.csv", index=False)